In [46]:
import stealth_requests as requests
import requests as r
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from dotenv import load_dotenv
import os
import json
load_dotenv()

True

In [47]:
urls = [
    "https://summer.skku.edu/summer/intro/welcome.do", # Welcome to SKKU Summer
    "https://summer.skku.edu/summer/intro/why.do", # Why Summer at SKKU?
    "https://summer.skku.edu/summer/program/overview.do", # Overview of SKKU Summer
    "https://summer.skku.edu/summer/program/2025Exclusive.do", #2025 Exclusive : Jeju Cultural Journey
    "https://summer.skku.edu/summer/program/activities.do", # Cultural Activity
    "https://summer.skku.edu/summer/life/Student_Service.do", # Student Services
    "https://summer.skku.edu/summer/program/policies.do", # Academic Policies
    ]

In [48]:
def scrape_content(url):
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the 'div' with class 'content-wrap'
    content_wrap_div = soup.find('div', class_='content-wrap')

    # Get the HTML content of this div
    if content_wrap_div:
        content_wrap_html = str(content_wrap_div)
        # print(content_wrap_html)
    else:
        print("Div with class 'content-wrap' not found.")

    md_content = md(content_wrap_html)
    print(md_content)


In [49]:
markdown_dict = {}
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        markdown_dict[url] = md(response.text)
    else:
        print(f"Failed to retrieve {url}: Status code {response.status_code}")

In [50]:
api_url = "http://localhost:8080/"
api_key = os.getenv("OPEN_WEBUI_API_KEY")
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

In [53]:
knowledge_base_data = {
    "name": "ISS Info",
    "description": "Knowledge Base with information about International Summer Semester",
    "data": {},
    "access_control": {}
}

# The api_url is defined in cell 4
create_kb_url = f"{api_url}api/v1/knowledge/create"
response = r.post(create_kb_url, headers=headers, json=knowledge_base_data)
if response.status_code == 200:
    print("Knowledge base created successfully.")
else:
    print(f"Failed to create knowledge base: {response.status_code} - {response.text}")

Knowledge base created successfully.
